In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

Comando abaixo é pra eu conseguir acessar meu drive do Google, porque programo usando o colab, que é uma plataforma deles pra programar em python online mesmo. Os dados dos pacientes estão salvos todos lá, então preciso de acesso

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + "Colab Notebooks/parkinson/"

**A parte principal do treinamento de qualquer modelo de machine learning é carregar os dados que farão nosso modelo 'aprender'. Os dados das imagens no nosso caso são armazenados em uma tabela: cada linha representa um paciente, e cada coluna representa uma medida que foi calculada a partir do exame (chamamos de característica, ou *feature*, na terminologia)**

In [5]:
#dfSpiral então é nossa tabela
dfSpiral = pd.read_csv(base_dir + 'NewSpiral.csv')

In [6]:
#head() imprime um pouco da tabela pra termos uma noção de como são os dados
dfSpiral.head()

,_ID_EXAM,IMAGE_NAME,ID_PATIENT,CLASS_TYPE,GENDER,RIGH/LEFT-HANDED,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
0,H1,sp1-H1.jpg,59,1,M,R,58,14900.772670,13207.241210,40856.39894,0.087041,56.978320,192.737170,0.026662,4394.307090,0.120773
1,H2,sp1-H2.jpg,76,1,M,R,60,6076.106534,9440.754502,36137.73814,0.065872,39.597318,169.753376,0.266595,2031.447046,0.172414
2,H3,sp1-H3.jpg,301,1,F,R,56,12220.462340,12247.263300,39871.73569,0.000000,46.183810,192.536776,0.007857,3843.754479,0.123894
3,H4,sp1-H4.jpg,102,1,M,R,24,4810.072073,4904.029458,32262.38975,0.044374,31.485790,200.401229,0.114530,1586.941716,0.162011
4,H5,sp1-H5.jpg,305,1,M,R,79,5458.875925,5911.591981,31795.46792,0.040906,47.231442,134.897863,0.026703,2251.866128,0.202899


In [7]:
#eliminamos algumas características não-numéricas abaixo
dfSpiral.drop(columns=['_ID_EXAM', 'IMAGE_NAME', 'ID_PATIENT', 'GENDER', 'RIGH/LEFT-HANDED'], inplace=True)

**Vamos visualizar de novo**

In [8]:
dfSpiral.head()

,CLASS_TYPE,AGE,RMS,MAX_BETWEEN_ET_HT,MIN_BETWEEN_ET_HT,STD_DEVIATION_ET_HT,MRT,MAX_HT,MIN_HT,STD_HT,CHANGES_FROM_NEGATIVE_TO_POSITIVE_BETWEEN_ET_HT
0,1,58,14900.772670,13207.241210,40856.39894,0.087041,56.978320,192.737170,0.026662,4394.307090,0.120773
1,1,60,6076.106534,9440.754502,36137.73814,0.065872,39.597318,169.753376,0.266595,2031.447046,0.172414
2,1,56,12220.462340,12247.263300,39871.73569,0.000000,46.183810,192.536776,0.007857,3843.754479,0.123894
3,1,24,4810.072073,4904.029458,32262.38975,0.044374,31.485790,200.401229,0.114530,1586.941716,0.162011
4,1,79,5458.875925,5911.591981,31795.46792,0.040906,47.231442,134.897863,0.026703,2251.866128,0.202899


**Eliminamos as colunas com features não-numéricas. Prosseguimos com a parte do treinamento. Usaremos um algoritmo clássico de machine learning chamado de Support Vector Machine. Esse vídeo explica a ideia das SVM's bem intuitivo: https://www.youtube.com/watch?v=Y6RRHw9uN9o**

In [10]:
#transformamos nossa tabela em outra que é exatamente igual, 
#só que o 'numpy' permite fazer operações com muito mais eficácia.
TRAIN = np.array(dfSpiral)

#abaixo separamos nossa matriz em características (parte numérica), 
#e rótulo (patológico/saudável)
X = TRAIN[:, 1:] #uma matriz com todas as features numéricas
Y = TRAIN[:, 0] #uma matriz com apenas 1 coluna (vetor), sendo essa coluna o diagnostico e cada paciente

X = StandardScaler().fit_transform(X) #normaliza features (importante no caso do algoritmo de aprendizado escolhido).
SVC = LinearSVC(dual=False)

scoresSVC = cross_val_score(SVC, X, Y, cv=9, scoring="accuracy") # testamos acurácias do nosso modelo

print(scoresSVC)

[0.73333333 0.96666667 0.8        0.75862069 0.68965517 0.82758621
 0.72413793 0.72413793 0.86206897]


In [ ]:
import joblib #importante isso aqui, serve pra gente exportar nosso modelo pro disco depois de treinar ele.

SVC.fit(X, Y) #aqui estamos falando para o nosso modelo de fato 'aprender' com nossos dados

filename = 'SVMSpiral.joblib'
joblib.dump(SVC, base_dir + filename)

**Pronto, modelo exportado. Quando formos fazer nossas previsões online, basta carregarmos esse arquivo. A mesma lógica é aplicada pros outros classificadores: realizamos o treinamento deles, e depois exportamos o modelo para o deploy.**